In [1]:
import ee
import geemap
import folium
import os

In [2]:
ee.Authenticate()
ee.Initialize(project='gee-map-id')

In [3]:
# Create an interactive map centered on Sri Lanka
Map = geemap.Map(center=[7.8731, 80.7718], zoom=7)

# Load Sri Lanka country boundary from FAO GAUL or similar dataset
sri_lanka = ee.FeatureCollection("FAO/GAUL/2015/level0") \
    .filter(ee.Filter.eq("ADM0_NAME", "Sri Lanka"))

# Load Sentinel-2 ImageCollection for Sri Lanka
image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(sri_lanka)
    .filterDate("2021-01-01", "2021-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .median()
    .clip(sri_lanka)  # Clip to the country boundary
    .select(["B2", "B3", "B4", "B8"])  # Blue, Green, Red, NIR
)

In [4]:
# Visualization parameters
vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B8", "B4", "B3"],  # NIR, Red, Green
}

Map.centerObject(sri_lanka, 7)
Map.addLayer(image, vis_params, "Sentinel-2")

In [5]:
# Create training data (example)
vegetation = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([80.7698, 7.8731]), {"landcover": 0}),
    ee.Feature(ee.Geometry.Point([80.7748, 7.8781]), {"landcover": 0}),
])

flat_lands = ee.FeatureCollection([  # Replaced 'water' with 'flat lands'
    ee.Feature(ee.Geometry.Point([80.7715, 7.8700]), {"landcover": 1}),
    ee.Feature(ee.Geometry.Point([80.7750, 7.8650]), {"landcover": 1}),
])

urban = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([80.7800, 7.8800]), {"landcover": 2}),
    ee.Feature(ee.Geometry.Point([80.7840, 7.8830]), {"landcover": 2}),
])

training_points = vegetation.merge(flat_lands).merge(urban)

In [6]:
# Extract training data from the image
bands = ["B2", "B3", "B4", "B8"]
label = "landcover"

training = image.select(bands).sampleRegions(
    collection=training_points,
    properties=[label],
    scale=10
)

In [7]:
# Train the classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50).train(
    features=training, classProperty=label, inputProperties=bands
)

# Apply the classifier to classify the image
classified = image.select(bands).classify(classifier)

# Create a folium map centered on Sri Lanka
folium_map = folium.Map(location=[7.8731, 80.7718], zoom_start=7)

# Add the classified image as a TileLayer (using geemap's get_map_id)
map_id = classified.getMapId({"min": 0, "max": 2, "palette": ["006400", "FFD700", "808080"]})  # Updated palette

# Add the classified layer to Folium map
folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr="Google Earth Engine",
    name="Land Cover Classification",
    overlay=True,
    control=True
).add_to(folium_map)

In [8]:
# Create a legend (custom HTML)
legend_html = '''
<div style="position: fixed;
            bottom: 30px; left: 30px; width: 160px; height: 120px;
            background-color: white; border:2px solid black; z-index:9999;
            font-size: 14px; padding: 10px;">
    <strong>Land Cover Legend</strong><br>
    <i style="background: #006400; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> Vegetation<br>
    <i style="background: #FFD700; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> Flat Lands<br>
    <i style="background: #808080; width: 20px; height: 20px; display: inline-block; margin-right: 5px;"></i> Urban<br>
</div>
'''

# Add the legend to the map
folium_map.get_root().html.add_child(folium.Element(legend_html))

In [9]:
# Add a layer control to the map
folium.LayerControl().add_to(folium_map)

# Save the map to an HTML file in the current directory
output_html = "D:/AgriMatrix/Area Calculation and Classification/sri_lanka_classified_map_with_legend_and_flatlands.html"
folium_map.save(output_html)

# Print the path to confirm where the map is saved
print(f"Map has been saved to {output_html}")

Map has been saved to D:/AgriMatrix/Area Calculation and Classification/sri_lanka_classified_map_with_legend_and_flatlands.html


In [10]:
folium_map